In [ ]:
import random
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf


path = "./vet_dataset_cleaned/"
test_perc = 0.15

# The meaning of life
random.seed(42)  
np.random.seed(42)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
import glob
import paths as paths


patient_paths = paths.get_patient_paths(path)
patient_paths.sort()

img_paths = [glob.glob(path + "/img/*") for path in patient_paths]
mask_paths = [glob.glob(path + "/mask/*") for path in patient_paths]

swap = int(len(img_paths) * test_perc)

print("Total:", swap + len(img_paths))
print("Training patients:", len(img_paths))
print("Testing patients:", swap)

In [ ]:
train_inputs = paths.flatten_list(img_paths[:-swap])
train_truths = paths.flatten_list(mask_paths[:-swap])

train_inputs.sort()
train_truths.sort()

test_inputs = paths.flatten_list(img_paths[-swap:])
test_truths = paths.flatten_list(mask_paths[-swap:])

test_inputs.sort()
train_truths.sort()

In [ ]:
shuffle_together = list(zip(train_inputs, train_truths))
random.shuffle(shuffle_together)
train_inputs, train_truths = zip(*shuffle_together)

train_inputs = np.array([np.load(array) for array in train_inputs])
train_truths = np.array([np.load(array) for array in train_truths])

print("Training inputs: ", train_inputs.shape)
print("Training truths: ", train_truths.shape)


data_mean = np.mean(train_inputs)
data_std = np.std(train_inputs)
print(data_mean)
print(data_std)

train_inputs = (train_inputs - data_mean) / data_std

In [ ]:


# _ = plt.hist(train_inputs.flatten(), bins=200)
# plt.xlim(-3, 3)

In [ ]:
test_inputs = paths.flatten_list(img_paths[-swap:])
test_truths = paths.flatten_list(mask_paths[-swap:])

test_inputs = np.array([np.load(array) for array in test_inputs])
test_truths = np.array([np.load(array) for array in test_truths])


print("Testing inputs: ", test_inputs.shape)
print("Testing truths: ", test_truths.shape)

test_inputs = (test_inputs - data_mean) / data_std

In [ ]:
index = 0 
perc_vacbag = np.sum(train_truths[...,index] > 0) / len(train_truths[...,index].flatten())
print(perc_vacbag)
print(1-perc_vacbag)

In [ ]:
def numberGenerator(n):
     number = 0
     while number < n:
         yield number
         number += 1

In [ ]:
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates

import cv2
import skimage.transform


def gaussian_noise(img, mean=0, sigma=0.003):
    img = img.copy()
    noise = np.random.normal(mean, sigma, img.shape)
    mask_overflow_upper = img+noise >= 1.0
    mask_overflow_lower = img+noise < 0
    noise[mask_overflow_upper] = 1.0
    noise[mask_overflow_lower] = 0
    img = img + noise
    return img

def random_crop_resize(img, label, crop_size=492):
    size_img = img.shape
    size_label = label.shape
    crop_size = random.randint(crop_size, img.shape[0]-1)
    crop_size = (crop_size, crop_size)

    # "Crop size should be less than image size"
    assert crop_size[0] <= img.shape[0] and crop_size[1] <= img.shape[1]

    w, h = img.shape[:2]
    x, y = np.random.randint(h-crop_size[0]), np.random.randint(w-crop_size[1])

    img = img[y:y+crop_size[0], x:x+crop_size[1],:]
    img = skimage.transform.resize(img, size_img)

    label = label[y:y+crop_size[0], x:x+crop_size[1],:]
    label = skimage.transform.resize(label, size_label)
    return img, label

def affine_transform(image, label, alpha_affine=0.5, random_state=None):

    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    shape_size = shape[:2]
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)

    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    image = image[...,np.newaxis]
    label = cv2.warpAffine(label, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    return image, label


def data_augment(img, mask, chance=1):
    # flip l/r
    if random.uniform(0,1) < chance:
        img = cv2.flip( img, 1 )
        mask = cv2.flip( mask, 1 )
        if len(img.shape) == 2:
            img = img[...,np.newaxis]
        if len(mask.shape) == 2:
            mask = mask[...,np.newaxis]

    # random crop and resize
    if random.uniform(0,1) < chance:
        img, mask = random_crop_resize(img, mask)
        if len(mask.shape) == 2:
            label = label[...,np.newaxis]

    # random affine transformation
    if random.uniform(0,1) < chance:
        img, mask = affine_transform(img, mask, alpha_affine=20)
        if len(img.shape) == 2:
            img = img[...,np.newaxis]
        if len(mask.shape) == 2:
            mask = mask[...,np.newaxis]
            
    # random gaussian noise
    if random.uniform(0,1) < chance:
        img = gaussian_noise(img)
        
    return img, mask


def read_array_list(arr_path_list):
    return np.array([np.load(arr_path) for arr_path in arr_path_list])


def data_generator(input_paths, truth_paths, batch_size, augment=False):
    batch_number = 0
    while batch_number < len(input_paths) / batch_size:
        batch_input_paths = input_paths[batch_number: batch_number + batch_size]
        batch_truth_paths = truth_paths[batch_number: batch_number + batch_size]
        batch_number += batch_size
        
        batch_imgs = []
        batch_masks = []
        
        for x, y in zip(batch_input_paths, batch_truth_paths):
            if augment is True:
                x = np.load(x)
                y = np.load(y)
                if random.uniform(0,1) < 0.5:
                    x, y = data_augment(x, y, chance=0.5)

            batch_imgs.append(x)
            batch_masks.append(y)
        
        yield np.array(batch_imgs), np.array(batch_masks)
        

In [ ]:
gen = data_generator(train_inputs, train_truths, 5, True)

In [ ]:
x, y = next(gen)

In [ ]:
plt.imshow(y[0,...,0])

# Training

In [ ]:
import train as train
import tensorflow as tf

INITIAL_LR = 1e-4
OPTIMIZER = tf.keras.optimizers.Adam(lr = INITIAL_LR)
LOSS = tf.keras.losses.BinaryCrossentropy()
WEIGHTS = "./vacbag/initial.hdf5"


model = train.compile_model(1, OPTIMIZER, LOSS, weights=WEIGHTS)

In [ ]:
import explore as explore
explore.plot_batch_predict(model, test_inputs, test_truths, x=0, batch_size = 2)

In [ ]:
MODEL_SAVE = "./vacbag/delete"
EPOCHS = 2
BATCH_SIZE = 8
LR_SCALE = 0.5
LR_PATIENCE = 3
STOP_PATIENCE = 100

model, train_history = train.train_model(model, train_inputs, train_truths, 
                                  MODEL_SAVE, 
                                  EPOCHS, 
                                  BATCH_SIZE, 
                                  LR_SCALE, 
                                  LR_PATIENCE, 
                                  STOP_PATIENCE)

In [ ]:
print(train_history.history)

train.plot_metric(train_history, '')

In [ ]:
#train.plot_loss(train_history)
explore.plot_batch_predict(model, test_inputs, test_truths, x=0, batch_size = 2)

In [ ]:
import loss as loss

INITIAL_LR = 1e-4
OPTIMIZER = tf.keras.optimizers.Adam(lr = INITIAL_LR)
LOSS = loss.dsc_loss
WEIGHTS = "./vacbag/bce_10_0.02_2020-04-29-15-53-02.hdf5"

model = train.compile_model(1, OPTIMIZER, LOSS, weights=WEIGHTS)

In [ ]:
MODEL_SAVE = "./vacbag/dice_after_bce"
EPOCHS = 1000
BATCH_SIZE = 5
LR_SCALE = 0.5
LR_PATIENCE = 3
STOP_PATIENCE = 100


train_history = train.train_model(model, train_inputs, train_truths, 
                                  MODEL_SAVE, 
                                  EPOCHS, 
                                  BATCH_SIZE, 
                                  LR_SCALE, 
                                  LR_PATIENCE, 
                                  STOP_PATIENCE)

In [ ]:
explore.plot_batch_predict(model, test_inputs, test_truths, x=0, batch_size = 5)

In [ ]:
eval_history = model.evaluate(x=test_inputs,
                        y=test_truths,
                        batch_size=5)

In [ ]:
train.plot_loss(eval_history)

In [ ]:
import loss as loss
import train as train

INITIAL_LR = 1e-6
OPTIMIZER = tf.keras.optimizers.Adam(lr = INITIAL_LR)
LOSS = loss.dsc_loss
WEIGHTS = "./vacbag/dice_after_bce_41_0.04_2020-04-29-16-21-27.hdf5"

model = train.compile_model(1, OPTIMIZER, LOSS, weights=WEIGHTS)


In [ ]:
eval_history = model.evaluate(x=test_inputs,
                        y=test_truths,
                        batch_size=1)

In [ ]:
import explore as explore
explore.plot_batch_predict(model, test_inputs, test_truths, x=5, batch_size = 5)

In [ ]:
plt.imshow(train_inputs[7,...,0])

In [ ]:
plt.imshow(train_truths[7,...,0])